In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# split a univariate sequence into samples
### sequence = (input variables, output)

In [ ]:

def split_sequences(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):            
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence):
            break
    # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix, :-1], sequence[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Transform 3d sequence in 2d sequence in order 'F'

In [ ]:
def ravel_sequence (inputseq):
    z = list()
    for s in range(len(inputseq)):
        l = np.ravel(inputseq[s],order='F')
        z.append(l)
    return z

# Trim Edge

In [ ]:
def trimEdge (dtset,n_excluded):
    dtset = dtset[n_excluded:]
    return dtset

In [ ]:
main_path = '../overfitting/setTraining/select_noOhe/TimeSeries_Prepared/'
train = np.load(main_path+'train.npy')
y_train = np.load(main_path+'y_train.npy')
print('shapes: train = {}, y_train = {}'.format(train.shape,y_train.shape))

In [ ]:
excluded = 3000 #exclued the first 'excluded' rows
train = trimEdge(train,excluded)
y_train = trimEdge(y_train,excluded)

In [ ]:
# standardscaler x data
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
X_trainScaled = sc_x.fit_transform(train)
dataset = np.concatenate((train,y_train),axis=1)
print("shape of dataset = {}".format(dataset.shape))

# Prepare Dataset

In [ ]:
#time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
X = ravel_sequence(X)
print("X.shape = {} , y.shape = {}".format(np.shape(X),np.shape(y)))

In [ ]:
# separate test dataset
percent_train = .8 #modify number for percent test dataset (1-percent_test)
percent = int(percent_train*len(X))
X_train = X[:percent]
X_test = X[percent:]
y_train = y[:percent]
y_test = y[percent:]
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

## MLP CLASSIFIER

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
mlp_rs = MLPClassifier(max_iter=500)
parameter_space = {
    'hidden_layer_sizes': [(10,12,10),(12,),(10,12,10,10),(12,12,12),(10,10,10,10,10),(256,128,64,32)],
    'activation': ['tanh', 'relu','logistic'],
    'solver': ['sgd', 'adam','lbfgs'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
    'batch_size':[10,50], 'power_t':[0.5],
}
clf = GridSearchCV(mlp_rs, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_train, y_train) # X is train samples and y is the corresponding labels

In [ ]:
print('Best parameters found:\n', clf.best_params_)

In [ ]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
y_true, y_pred = y_test , clf.predict(X_test)
from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))